In [19]:
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
import pandas as pd
from kerastuner.tuners import BayesianOptimization, Hyperband
from sklearn.preprocessing import MinMaxScaler

In [20]:
def replace_carType(carType):
    try:
        types = ['van', 'regcar', 'sportuv', 'sportcar', 'stwagon', 'truck']
        return types.index(carType)
    except ValueError:
        return carType


def replace_fuelType(fuelType):
    try:
        types = ['cng', 'methanol', 'electric', 'gasoline']
        return types.index(fuelType)
    except ValueError:
        return fuelType

In [21]:
path = 'trainingData.csv'
df = read_csv(path)

for col in df.columns[5:11]:
    df[col] = df[col].apply(replace_carType)
for col in df.columns[11:17]:
    df[col] = df[col].apply(replace_fuelType)

df.drop(['id'], axis=1, inplace=True)
X, y = df.values[:, 1:], df.values[:, 0]

X = X.astype('float32')
y = LabelEncoder().fit_transform(y)

In [22]:
scaler = MinMaxScaler()
arr_scaled = scaler.fit_transform(X)
X = pd.DataFrame(arr_scaled)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

n_features = X_train.shape[1]


In [23]:
def create_model(hp):
    model = Sequential()
    for i in range(hp.Int('layers', 2, 15)):
        model.add(Dense(units=hp.Int('units_' + str(i), min_value=10, max_value=512, step=16),
                        activation=hp.Choice('act_' + str(i), ['relu', 'tanh', 'sigmoid']),
                        kernel_initializer='he_normal', input_shape=(n_features,)))

    model.add(Dense(6, activation='softmax'))

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

In [24]:
bayesian_tuner = BayesianOptimization(create_model, objective='val_accuracy', max_trials=1, project_name='BayesianOptimization')

hyperBand_tuner = Hyperband(create_model, objective='val_accuracy', max_epochs=1, hyperband_iterations=2, project_name='Hyperband')

In [25]:
bayesian_tuner.search_space_summary()

Search space summary
Default search space size: 5
layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 15, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 10, 'max_value': 512, 'step': 16, 'sampling': None}
act_0 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'sigmoid'], 'ordered': False}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 10, 'max_value': 512, 'step': 16, 'sampling': None}
act_1 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'sigmoid'], 'ordered': False}


In [26]:
hyperBand_tuner.search_space_summary()

Search space summary
Default search space size: 5
layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 15, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 10, 'max_value': 512, 'step': 16, 'sampling': None}
act_0 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'sigmoid'], 'ordered': False}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 10, 'max_value': 512, 'step': 16, 'sampling': None}
act_1 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'sigmoid'], 'ordered': False}


In [27]:
bayesian_tuner.search(X_train, y_train, epochs=40, validation_data=(X_test, y_test))

Trial 1 Complete [00h 00m 26s]
val_accuracy: 0.39387890696525574

Best val_accuracy So Far: 0.39387890696525574
Total elapsed time: 00h 00m 26s
INFO:tensorflow:Oracle triggered exit


In [28]:
bayesian_tuner.results_summary()

Results summary
Results in .\BayesianOptimization
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
layers: 12
units_0: 170
act_0: sigmoid
units_1: 378
act_1: tanh
units_2: 10
act_2: relu
units_3: 10
act_3: relu
units_4: 10
act_4: relu
units_5: 10
act_5: relu
units_6: 10
act_6: relu
units_7: 10
act_7: relu
units_8: 10
act_8: relu
units_9: 10
act_9: relu
units_10: 10
act_10: relu
units_11: 10
act_11: relu
Score: 0.39387890696525574


In [31]:
bayesian_tuner.get_best_models(num_models=5)

In [35]:
best_param = bayesian_tuner.get_best_hyperparameters()[:5]

for i in best_param:
    print(i.values, '\n')

{'layers': 12, 'units_0': 170, 'act_0': 'sigmoid', 'units_1': 378, 'act_1': 'tanh', 'units_2': 10, 'act_2': 'relu', 'units_3': 10, 'act_3': 'relu', 'units_4': 10, 'act_4': 'relu', 'units_5': 10, 'act_5': 'relu', 'units_6': 10, 'act_6': 'relu', 'units_7': 10, 'act_7': 'relu', 'units_8': 10, 'act_8': 'relu', 'units_9': 10, 'act_9': 'relu', 'units_10': 10, 'act_10': 'relu', 'units_11': 10, 'act_11': 'relu'} 

